In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [40, 3]
plt.figure(figsize=(15,15))


In [ ]:
df = pd.read_csv('KO_vsWT.tsv', sep='\t', index_col='Geneid')
subdf = df[df.PValue.le(.05) & df.FDR.le(.01)].copy()
subdf['log_pval'] = np.log(subdf.PValue) *-1
annotation = pd.read_csv('nk_atac_peaks.out', sep='\t', index_col=0)
adf = pd.merge(annotation, df, left_index=True, right_index=True)
sel_g = pd.read_csv('select_g.csv')
sel_g = list(set(sel_g.genes))

In [ ]:
de_mef = pd.read_csv('sva_mef_cleaned.csv', index_col=0)
de_adp = pd.read_csv('sva_adipose_cleaned.csv', index_col=0)


In [ ]:
sub_ad = adf[adf['Gene Name'].isin(de_mef.index) & adf.Annotation.isin(['Intergenic','promoter-TSS'])].copy()
sub_ad = sub_ad.merge(de_mef,left_on='Gene Name', right_index=True)
sub_ad['gene'] = sub_ad['Gene Name'].copy()
cad = sub_ad.drop(columns=['Chr_y','Start_y','End_y','Chr_x','Start_x','End_x','Focus Ratio/Region Size','Nearest PromoterID','Entrez ID',
                     'Nearest Unigene','Nearest Refseq','Nearest Ensembl','Gene Description','Gene Name','Strand'])
sub_ad = cad[cad.pvalue.le(.05) & cad.PValue.le(.05)]
sub_ad['match'] = (sub_ad.log2FoldChange.gt(0) & sub_ad.logFC.gt(0)) |(sub_ad.log2FoldChange.lt(0) & sub_ad.logFC.lt(0))
sub_mef = sub_ad.copy()
sub_mef['selected'] = sub_mef.gene.isin(sel_g)

In [ ]:
sub_mef.match = ~sub_mef.match

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import numpy as np
from adjustText import adjust_text

# Define a consistent palette for the 'match' hue
unique_matches = sub_mef['match'].unique()
unique_matches = unique_matches[::-1]
palette = sns.color_palette("Set2", n_colors=len(unique_matches))

plt.figure(figsize=(25, 25))

# Create scatterplot with both hue and style
ax = sns.scatterplot(data=sub_mef, x='log2FoldChange', y='logFC',
                     hue='match',
                     palette=palette, legend='full')

# Remove the automatically generated legend (which includes both hue and style)
if ax.legend_ is not None:
    ax.legend_.remove()

# Manually create a legend for the hue ('match') only
patches = [mpatches.Patch(color=palette[i], label=match) for i, match in enumerate(unique_matches)]
plt.legend(handles=patches, title="match", fontsize=16, title_fontsize=18)

# Plot text labels for genes, increasing font size and storing texts for adjustment
texts = []
tml = sub_mef.sort_values('logFC', ascending=False).head(10).index.tolist() + sub_mef.sort_values('logFC', ascending=True).head(10).index.tolist()
tml += sub_mef.sort_values('log2FoldChange', ascending=False).head(10).index.tolist() + sub_mef.sort_values('log2FoldChange', ascending=True).head(10).index.tolist()
for i in tml:
    # Only label points with sufficient magnitude
    
    # if (np.abs(sub_mef.loc[i].logFC) < 1) or (np.abs(sub_mef.loc[i].log2FoldChange) < 1):
    #     continue
    texts.append(plt.text(x=sub_mef.log2FoldChange.loc[i] ,
                          y=sub_mef.logFC.loc[i] ,
                          s=sub_mef.gene.loc[i],
                          fontdict={'size': 40, 'weight':'bold'}))

# Adjust text positions to reduce overlap with more iterations and modified parameters
adjust_text(texts,
            autoalign='xy',
            only_move={'texts': 'xy'},
            force_text=5,
            force_points=0,
            expand_text=(15, 15),
            expand_points=(0, 0),
            arrowprops=dict(arrowstyle='->', color='gray'),
            lim=100000)

# Update title and axis labels with larger fonts
plt.title("MEF KO vs WT :: ATAC vs RNA", fontsize=24)
plt.xlabel("log2FoldChange", fontsize=16)
plt.ylabel("log2FoldChange", fontsize=16)

plt.tight_layout()
plt.savefig('MEF_atac_vs_rna_b.pdf')
plt.show()


In [ ]:
plt.figure(figsize=(5,5))

from matplotlib_venn import venn2
mef_g = de_mef[de_mef.padj.le(.05)].index.tolist()
adp_g = de_adp[de_adp.padj.le(.05)].index.tolist()
shared_g = set(mef_g) & set(adp_g)
venn2(subsets = (len(mef_g) - len(shared_g), len(adp_g) - len(shared_g), len(shared_g)), set_labels = ('MEF', 'VAT') )
plt.title(f"shared DE genes WT vs KO \n padj <= .05 \nMEF & VAT :: total {len(mef_g)  + len(adp_g) - len(shared_g) }")
plt.savefig('MEF_VAT_DGE_venn.pdf')


In [ ]:
# shared_g
out_a = sub_adp[['logFC','PValue','FDR','log2FoldChange','pvalue','padj','gene','match','selected']]
out_m = sub_mef[['logFC','PValue','FDR','log2FoldChange','pvalue','padj','gene','match','selected']]
out_a['sample'] = 'ADP'
out_m['sample'] = 'MEF'

In [ ]:
DE_both = pd.concat([out_m,out_a])

In [ ]:
DE_both.columns = ['logFC_ATAC', 'PValue', 'FDR', 'log2FoldChange_RNA', 'pvalue', 'padj', 'gene', 'ATAC_RNA_FC_match', 'selected', 'sample']


In [ ]:
import scanpy as sc 

In [ ]:
adata = sc.read_10x_mtx('Downloads/GSE264266_RAW/')
adata
# adata.var

In [ ]:
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata.layers['counts'] = adata.X.copy()
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=10)
adata.var['mt'] = adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")
adata = adata[adata.obs.pct_counts_mt.le(15)].copy()
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")


In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
sc.pp.scale(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=.3, key_added='undercluster')

In [ ]:
sc.pl.umap(adata, color=['undercluster','Smpd3','Syt7'], legend_loc='on data')

In [ ]:
sc.pl.pca_loadings(adata)

In [ ]:
priors = adf[adf['Gene Name'].isin(list(set(shared_g) & set(adata.var_names))) & ~adf.Annotation.str.startswith('intron')][adf.columns[0:3]]


In [ ]:
priors.to_csv('Desktop/adp_priors.bed', sep='\t', index=False, header=False)

In [ ]:
adata.X = adata.layers['counts'].copy()
adata.write_h5ad('Desktop/SAMIRA/subnet/input.h5ad')

In [ ]:
open('Desktop/SAMIRA/subnet/tf_list.txt','w').write('\n'.join(adata[:,adata.var_names.isin(tf.Symbol.tolist())].var_names.tolist()))

In [ ]:
priors = pd.read_csv('Desktop/adp_priors.bed', sep='\t')


In [ ]:
import scanpy as sc
import pandas as pd
adata = sc.read_h5ad('Desktop/SAMIRA/subnet/input.h5ad')


In [ ]:
net = pd.read_csv('/Volumes/GS4T/WORK/SAMIRA/subnet_out_tfa/network.tsv.gz', sep='\t')

In [ ]:
coef = pd.read_csv('/Volumes/GS4T/WORK/SAMIRA/subnet_out/model_coefficients.tsv.gz', sep='\t', index_col=0)

In [ ]:
DE_both = pd.read_csv('Desktop/shared_MEF_ADP.txt', header=None)

In [ ]:
sel_g = pd.read_csv('/Volumes/GS4T/WORK/MAY_2024/SAMIRA/select_g.csv')
sel_g = list(set(sel_g.genes))
subnet = net[net.target.isin(DE_both[0]) | net.target.isin(sel_g)].copy()

In [ ]:
subnet[subnet['model_exp_var'].ge(.1)]

In [ ]:
net[net.target.isin(sel_g) & net.model_exp_var.gt(.01)].target.value_counts()

In [ ]:
net[net.target.isin(sel_g) & net.model_exp_var.gt(.01)]

In [ ]:
my_net = set(net[net.target.isin(sel_g)].target.unique().tolist() + net[net.target.isin(sel_g)].regulator.unique().tolist())

In [ ]:
adata.X = adata.layers['counts'].copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sub_ad = adata[:,list(my_net)].copy()
my_net = set(net[net.target.isin(sel_g)].target.unique().tolist() + net[net.target.isin(sel_g)].regulator.unique().tolist())


In [ ]:
infer = sc.read_h5ad('/Volumes/GS4T/WORK/SAMIRA/subnet_out_tfa/inferelator_model.h5ad')

In [ ]:
# priors = pd.read_csv('/Volumes/GS4T/WORK/SAMIRA/subnet/sam_priors_edge_matrix.tsv', sep='\t', index_col=0)
# priors
sc.pp.log1p(adata)
sc.pp.scale(adata)
# adata.X
sub_ad = adata[:,list(my_net)].copy()
sub_ad

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Example: Generating synthetic data for demonstration
np.random.seed(42)





# Parameters for ElasticNet
param_grid = {
    'elasticnet__alpha': np.logspace(-3, -1, 50),  # Alpha range
    'elasticnet__l1_ratio': [0.01, 0.1, 0.2, 0.25,.3,.35,.4]  # L1 ratio range
}

# Results storage
results = []
models = {}
adata.X = adata.layers['counts'].copy()
sc.pp.normalize_total(adata, target_sum=1e4)
my_net = set(net[net.target.isin(sel_g)].target.unique().tolist() + net[net.target.isin(sel_g)].regulator.unique().tolist())
sub_ad = adata[:,list(my_net)].copy()

# Loop over 200 regression tasks
i=0
for x in sel_g:
    my_regs = net[net.target.eq(x)].regulator.tolist()
    if len(my_regs)==0:
        continue
    X = adata[:,my_regs].X.toarray()
    y = adata[:,x].X.toarray()    
    # Train-test split for this task
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Pipeline with scaling and ElasticNet
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Scaling
        ('elasticnet', ElasticNet(random_state=42, fit_intercept=False))  # ElasticNet without intercept
    ])

    # GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Save results
    results.append({
        'Task': i + 1,
        'Gene':x,
        'Best Alpha': grid_search.best_params_['elasticnet__alpha'],
        'Best L1 Ratio': grid_search.best_params_['elasticnet__l1_ratio'],
        'Mean Squared Error': mse,
        'R-squared': r2,
    })
    models[x] = best_model.named_steps['elasticnet']
    i+=1
# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
# import ace_tools as tools; tools.display_dataframe_to_user(name="ElasticNet Regression Results for MEF", dataframe=results_df)


In [ ]:
# net[net.target.eq('Vegfa')].regulator.tolist()

tmp_v = pd.DataFrame(models[my_f_g].coef_, index=net[net.target.eq(my_f_g)].regulator.tolist())
tmp_v = tmp_v[np.abs(tmp_v[0])>0]
tmp_v

In [ ]:
net[net.target.eq(my_f_g) & net.regulator.isin(tmp_v.index)]

In [ ]:
# bulk_mef = pd.read_csv('Desktop/SAMIRA/subnet_out/MEF_bulk.csv', index_col=0)
WT = bulk_mef.iloc[0:3]
WT

In [ ]:
WT[[x for x in WT.columns if x in sel_g]]
# WT[['Vegfa', 'Shox2', 'Fos', 'Jun']]
# WT[['Lst1','Vezf1', 'Fosl1']]

In [ ]:
scaler = StandardScaler()
# scaled_matrix = scaler.fit_transform(WT[['Vegfa', 'Shox2', 'Fos', 'Jun']])
scaled_matrix = scaler.fit_transform(WT[[my_f_g]+tmp_v.index.tolist()])
scaled_matrix

In [ ]:
# rg = pd.DataFrame(scaled_matrix, columns=['Vegfa', 'Shox2', 'Fos', 'Jun'])
# rg[['Shox2', 'Fos', 'Jun']].dot(tmp_v[0].tolist()) 
rg = pd.DataFrame(scaled_matrix, columns=[my_f_g]+tmp_v.index.tolist())
print(rg[tmp_v.index.tolist()].dot(tmp_v[0].tolist()) - rg[my_f_g])
print(rg[tmp_v.index.tolist()].dot(tmp_v[0].tolist()))
print(rg[my_f_g])
# rg.dot(tmp_v[0].tolist()) 
# print(tmp_v.index.tolist())


In [ ]:
net[net.target.isin(sel_g)].to_csv('Desktop/SAMIRA/subnet_out/filt_net.tsv', sep='\t', index=None)

In [ ]:
TFA = pd.read_csv('Desktop/SAMIRA/subnet_out_tfa/TFA.tsv', sep='\t')
TFA

In [ ]:
# infer.uns['network']
tfanet = net[net.target.isin(sel_g) & net.regulator.isin(TFA.columns)].copy()
tfanet = tfanet[['target','regulator','model_coefficient']].copy()

In [ ]:
tfanet
TFA = TFA.set_index('Unnamed: 0')


In [ ]:
import pandas as pd
# Example: betas DataFrame (regulator, target, model_coefficient)
# betas = pd.DataFrame({
#     "target": ["GeneA", "GeneB", "GeneA", "GeneC"],
#     "regulator": ["TF1", "TF1", "TF2", "TF3"],
#     "model_coefficient": [0.5, -0.3, 1.2, 0.8]
# })

# Example: TFA DataFrame (observations x regulators)
# TFA = pd.DataFrame({
#     "TF1": [0.2, 0.4],
#     "TF2": [0.1, 0.3],
#     "TF3": [0.5, 0.7]
# })

# Step 1: Pivot the betas DataFrame into a matrix
B = tfanet.pivot(index="regulator", columns="target", values="model_coefficient").fillna(0)

# Step 2: Filter to keep only regulators present in TFA.columns
common_regulators = B.index.intersection(TFA.columns)
B = B.loc[common_regulators]

In [ ]:
# result = TFA @ B  # Matrix multiplication
# TFA
B

In [ ]:
# TFA.columns[TFA.columns.isin(B.index)].tolist()
rebuilt_expression = TFA.loc[:,TFA.columns[TFA.columns.isin(B.index)].tolist()] @ B

In [ ]:
rebuilt_expression

In [ ]:
adata[rebuilt_expression.index.tolist(), rebuilt_expression.columns.tolist()].X.todense() - rebuilt_expression

In [ ]:
# compute the activity from the bulk replicates 
# then mux it, refit the coefficients and predict the KO expression 

adata[rebuilt_expression.index.tolist(), rebuilt_expression.columns.tolist()]

In [ ]:
net.loc[tfanet.index][net.loc[tfanet.index]['model_exp_var'].gt(.05)].regulator.value_counts()

In [ ]:
df = sub_mef[sub_mef.gene.isin(['Npas4',
'Clock',
'E2f8',
'Zeb2',
'Smad5',
'Lef1',
'Klf6',
'Foxo3',
'Sox4',
'Glis2',
'Sox2',
'Pbx2',
'Maz',
'Klf16',
'Barx1',
'Tbx2',
'Sox13',
'Sox8',
'Gata4'])][['logFC','gene']]

heatmap_data = df.pivot(columns='gene', values='logFC')

In [ ]:
# heatmap_data = df.pivot(index='sample', columns='gene', values='logFC')

# Plot heatmap
plt.figure(figsize=(8, 10))
plt.imshow(heatmap_data.values, aspect='auto')
plt.colorbar(label='logFC')
plt.xticks(range(len(heatmap_data.columns)), heatmap_data.columns, rotation=90)
plt.yticks(range(len(heatmap_data.index)), heatmap_data.index)
plt.xlabel('Gene')
plt.ylabel('Peak')
plt.title('Heatmap of logFC by Gene and peaks')
plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.colors import TwoSlopeNorm

norm = TwoSlopeNorm(vmin=heatmap_data.values.min(),
                    vcenter=0,
                    vmax=heatmap_data.values.max())

plt.figure(figsize=(8, 10))
plt.imshow(heatmap_data.values, aspect='auto',
           cmap='bwr', norm=norm)
plt.colorbar(label='logFC')
plt.xticks(range(len(heatmap_data.columns)),
           heatmap_data.columns, rotation=90)
plt.yticks(range(len(heatmap_data.index)),
           heatmap_data.index)
plt.xlabel('Gene')
plt.ylabel('Sample')
plt.title('log2FC accessibility of peaks by genes  WT vs KO')
plt.tight_layout()
plt.show()

In [ ]:
heatmap_data.to_csv('Desktop/MEF_NM1_ATAC_FC.csv')